In [8]:
import pandas as pd
import pandas as pd

path1 = "/Users/dear/Data Science/Project/Data-Sci-project/results/data_formatted_subjectAreas.csv"
path2 = "C:/Users/PangSunatcha/OneDrive - Chulalongkorn University/Documents/Y2S1 files/Data Sci/proj/Data-Sci-project/results/data_formatted_subjectAreas.csv"
df = pd.read_csv(path2)
df["Processed Words"]

0        ['recent', 'developments', 'bifunctional', 'ai...
1        ['benefit', 'punishment', 'sensitivity', 'moto...
2        ['magic', 'biomarkers', 'predict', 'longterm',...
3        ['endocrine', 'disrupting', 'compounds', 'envi...
4        ['probabilistic', 'regular', 'grammar', 'infer...
                               ...                        
20211    ['prediction', 'diagnosis', 'chronic', 'kidney...
20212    ['incidence', 'healthcareassociated', 'urinary...
20213    ['turmericloaded', 'alginate', 'particulatebas...
20214    ['identification', 'novel', 'gut', 'microbiota...
20215    ['effectiveness', '1', '2per', 'cent', 'acetic...
Name: Processed Words, Length: 20216, dtype: object

In [9]:
import ast

df['Processed Words'] = df['Processed Words'].apply(ast.literal_eval)
df['Author Keywords'] = df['Author Keywords'].fillna('')
df['combined keywords'] = df.apply(
    lambda row: row['Processed Words'] + row['Author Keywords'].split(', '),
    axis=1
)
df['combined keywords']

0        [recent, developments, bifunctional, air, elec...
1        [benefit, punishment, sensitivity, motor, perf...
2        [magic, biomarkers, predict, longterm, outcome...
3        [endocrine, disrupting, compounds, environment...
4        [probabilistic, regular, grammar, inference, a...
                               ...                        
20211    [prediction, diagnosis, chronic, kidney, disea...
20212    [incidence, healthcareassociated, urinary, tra...
20213    [turmericloaded, alginate, particulatebased, b...
20214    [identification, novel, gut, microbiota, signa...
20215    [effectiveness, 1, 2per, cent, acetic, acid, s...
Name: combined keywords, Length: 20216, dtype: object

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


# Prepare features and labels
X = df['combined keywords'].apply(lambda x: ' '.join(x))
y = df['Subject Areas']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF feature extraction
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Logistic Regression model
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = logreg.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.4735410484668645

Classification Report:
                                       precision    recall  f1-score   support

Agricultural and Biological Sciences       0.52      0.55      0.53       245
                 Arts and Humanities       0.62      0.14      0.22        37
                        Biochemistry       0.24      0.19      0.21       219
                            Business       0.47      0.22      0.30        67
                Chemical Engineering       0.45      0.25      0.32       119
                           Chemistry       0.32      0.35      0.33       241
                    Computer Science       0.52      0.61      0.56       257
                   Decision Sciences       0.00      0.00      0.00         4
                           Dentistry       0.53      0.30      0.39        53
        Earth and Planetary Sciences       0.63      0.20      0.30        60
            Econometrics and Finance       0.29      0.05      0.09        38
         

c:\Users\PangSunatcha\.conda\envs\DSenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PangSunatcha\.conda\envs\DSenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PangSunatcha\.conda\envs\DSenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Load the dataset
# df = pd.read_csv("/path/to/data_formatted_subjectAreas.csv")
# df['Processed Words'] = df['Processed Words'].apply(eval)

# # Prepare features and labels
# X = df['Processed Words'].apply(lambda x: ' '.join(x))
# y = df['Subject Areas']

# # Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # TF-IDF feature extraction
# tfidf = TfidfVectorizer(max_features=5000)
# X_train_tfidf = tfidf.fit_transform(X_train)
# X_test_tfidf = tfidf.transform(X_test)

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [100, 500, 1000]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Train the model
grid_search.fit(X_train_tfidf, y_train)

# Best parameters and evaluation
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Evaluate the model
y_pred = best_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}
Accuracy: 0.4735410484668645

Classification Report:
                                       precision    recall  f1-score   support

Agricultural and Biological Sciences       0.52      0.55      0.53       245
                 Arts and Humanities       0.62      0.14      0.22        37
                        Biochemistry       0.25      0.19      0.21       219
                            Business       0.47      0.22      0.30        67
                Chemical Engineering       0.45      0.25      0.32       119
                           Chemistry       0.32      0.35      0.33       241
                    Computer Science       0.51      0.60      0.56       257
                   Decision Sciences       0.00      0.00      0.00         4
                           Dentistry       0.53      0.30      0.39        53
        Earth and Planeta

c:\Users\PangSunatcha\.conda\envs\DSenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PangSunatcha\.conda\envs\DSenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PangSunatcha\.conda\envs\DSenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu